In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys
import time

In [2]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [4]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
device = torch.device("cuda:1")
#device = torch.device("cpu")

In [7]:
model_name = 'flow_model_weighted_max_best_validation'
samples_name = model_name.replace('model', 'samples')
llhs_name = model_name.replace('model', 'llhs')

In [8]:
flow = torch.load(model_name+".pt", map_location=device)
flow.eval()

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): RandomPermutation()
      (1): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(in_features=14, out_features=100, bias=True)
          (blocks): ModuleList()
          (final_layer): MaskedLinear(in_features=100, out_features=196, bias=True)
        )
      )
      (2): RandomPermutation()
      (3): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(in_features=14, out_features=100, bias=True)
          (blocks): ModuleList()
          (final_layer): MaskedLinear(in_features=100, out_features=196, bias=True)
        )
      )
      (4): RandomPermutation()
      (5): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(in_features=14, out_features=100, bias=Tru

In [9]:
t_start = time.time()

In [10]:
n_samples = int(1e7)
n_sample_size = int(1e5)

steps = int(n_samples/n_sample_size)

generated_samples = None
generated_llhs = None
for i in range(steps):
    print("Sample {} points ({}/{})...".format(n_sample_size, i+1, steps))
    with torch.no_grad():
        new_data = flow.sample(n_sample_size)
        new_llhs = np.exp(flow.log_prob(new_data).cpu().numpy())
        new_data = new_data.cpu().numpy()
        if generated_samples is None:
            generated_samples = new_data
            generated_llhs = new_llhs
        else:
            generated_samples = np.vstack((generated_samples, new_data))
            generated_llhs = np.vstack((generated_llhs, new_llhs))

Sample 100000 points (1/100)...
Sample 100000 points (2/100)...
Sample 100000 points (3/100)...
Sample 100000 points (4/100)...
Sample 100000 points (5/100)...
Sample 100000 points (6/100)...
Sample 100000 points (7/100)...
Sample 100000 points (8/100)...
Sample 100000 points (9/100)...
Sample 100000 points (10/100)...
Sample 100000 points (11/100)...
Sample 100000 points (12/100)...
Sample 100000 points (13/100)...
Sample 100000 points (14/100)...
Sample 100000 points (15/100)...
Sample 100000 points (16/100)...
Sample 100000 points (17/100)...
Sample 100000 points (18/100)...
Sample 100000 points (19/100)...
Sample 100000 points (20/100)...
Sample 100000 points (21/100)...
Sample 100000 points (22/100)...
Sample 100000 points (23/100)...
Sample 100000 points (24/100)...
Sample 100000 points (25/100)...
Sample 100000 points (26/100)...
Sample 100000 points (27/100)...
Sample 100000 points (28/100)...
Sample 100000 points (29/100)...
Sample 100000 points (30/100)...
Sample 100000 point

In [11]:
t_end = time.time()
timespan = t_end - t_start
print("Time to sample {} datapoints: {}".format(n_samples, timespan))

Time to sample 10000000 datapoints: 176.89814019203186


In [12]:
np.savetxt('data/{}.csv'.format(samples_name), generated_samples, delimiter=',')
np.savetxt('data/{}.csv'.format(llhs_name), generated_llhs.reshape(-1,1), delimiter=',')